# Filtering

FILES

https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_absence.txt
https://raw.githubusercontent.com/ScryptIQ-ai/lesson-data/refs/heads/main/dh/EEG_background.txt

PACKAGES

pandas
matplotlib
numpy
scipy

## Introduction

**Learning Objectives**
- Learn how to implement a band-pass filter

Datasets with complex waveforms contain many different components which may or may not be relevant to a specific question. In such situations it can be useful to filter your data, ensuring that you are removing specific components from the dataset that are not relevant to your analyses or question. In this context, the term component refers to 'frequency', i.e. the number of cycles the waveform completes per unit of time. A small number refers to low frequencies with long periods (cycles), and a large number refers to high frequencies with short periods.

## Band-pass filter

Let's explore a simple example, demonstrating how both low- and high-frequency components can be filtered (suppressed) in our example time series.

Let's begin by defining a simple function which takes two additional input arguments: low and high cut-off.

In [ ]:
from scipy.signal import butter, sosfilt

In [ ]:
def data_filter(data, sr, low, high):
    """
    Filtering of multiple time series.
    
    data: nxm numpy array. Rows are time points, columns are recordings
    sr: sampling rate, same time units as period
    low: Low cut-off frequency (high-pass filter)
    high: High cut-off frequency (low-pass filter)
    
    return: filtered data
    """

    
    order = 5
    filter_settings = [low, high, order]
    
    sos = butter(order, (low,high), btype='bandpass', fs=sr, output='sos')
    
    data_filtered = zeros((data.shape[0], data.shape[1]))
    
    for index, column in enumerate(data.transpose()): # We transpose (flip the axis) the data to get columns as rows
        forward = sosfilt(sos, column)
        backwards = sosfilt(sos, forward[-1::-1])
        data_filtered[:, index] = backwards[-1::-1]
    
    return data_filtered

In [ ]:
df_back = read_csv("./data/EEG_background.txt", sep=r"\s+")
df_epil = read_csv("./data/EEG_absence.txt", sep=r"\s+")

d1 = df_back.to_numpy()
d2 = df_epil.to_numpy()

sr = 256      # 1 / seconds
period = 6    # seconds
channels = 10

data_back = d1[:period*sr:, :channels]
data_epil = d2[:period*sr:, :channels]

NOTE BOX

We have loaded in plot_series() in the background to prevent visual clutter.

In [ ]:
data_back_filt = data_filter(data_back, sr, 8, 13)
names = df_back.columns[:channels]

(fig, ax) = plot_series(data_back_filt, sr)

fig.suptitle('Filtered Recording of Background EEG', fontsize=16)
legend(names)

show()

The frequency range from 8 to 13 Hz is referred to as alpha band in our EEG. It is thought that this represents a type of idling rhythm in the brain where the brain is not actively processing sensory input.

PRACTICE EXERCISE

Create figures of the delta (1-4 Hz) band for both the background and the seizure EEG. Note the differences.

In [ ]:
# Write you code here:

## Summary

Implementing band-pass filters that selectively suppress unwanted frequency components allows us to isolate specific oscillatory patterns within complex time series data. Understanding how to define cut-off frequencies and apply bidirectional filtering ensures that only the frequency ranges relevant to a particular research question are retained for analysis. This approach enables targeted investigation of distinct physiological rhythms, such as the alpha or delta bands in EEG recordings, revealing patterns that would otherwise be obscured by broadband noise. Mastering frequency-domain filtering facilitates cleaner signal extraction, more accurate pattern recognition and robust quantitative comparisons across experimental conditions or clinical states.